In [ ]:
%cd ~
%cd ..
%cd ..
%cd "Volumes/TOSHIBA EXT"
%cd "Data_Analytics_Data"

/Users/xiongh
/Users
/
[Errno 2] No such file or directory: 'Volumes/TOSHIBA EXT'
/
[Errno 2] No such file or directory: 'Data_Analytics_Data'
/


In [ ]:
%ls

CCHF - Total ProMED Archive.xlsx  Daily_GPM_Data_0.25_Resolution/


In [ ]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import netCDF4
import netcdftime
import datetime
import os
import time

In [ ]:
def fill_precip_na(_df):
    _df = _df.copy()
    _df.fillna(method='ffill',inplace = True)
    _df.fillna(method='bfill',inplace = True)
    return _df

In [ ]:
def nc4_to_df(meta_data):
    '''
    convert a single nc4 file to pandas df including four columns
    time, longitude, latitude, and precipitation 
    '''
    # meta_data = '/content/data/3B-DAY.MS.MRG.3IMERG.20160422-S000000-E235959.V06.nc4'

    nc_fid = netCDF4.Dataset(meta_data, "r", format="NETCDF4")


    _time = nc_fid.BeginDate
    #t_unit = nc_fid.variables['time'].units
    # t_cal = nc_fid.variables['time'].calendar

    #nc4date = netcdftime.num2date(time,units = t_unit,calendar = 'Gregorian')[0]
    #date_time = datetime.datetime(nc4date.year,nc4date.month,nc4date.day)

    date_str = pd.to_datetime(_time).strftime("%m/%d/%Y")
    #print("date and time:",date_str)

    lats = nc_fid.variables['lat'][:]  # extract/copy the data
    lons = nc_fid.variables['lon'][:]
    _time = nc_fid.BeginDate

    precip = nc_fid.variables['precipitation'][:]

    latitudes_grid, longitudes_grid = [
      x.flatten() for x in np.meshgrid(lats, lons, indexing='ij')]
    df = pd.DataFrame({
      'lat': latitudes_grid,
      'lon': longitudes_grid,
      'precip': precip[:].flatten()})
    length = len(df)
    time_seris = pd.Series([date_str]*length)
    df['time'] = time_seris
    df = df.copy()

    return df

In [ ]:
# Filter out the loaction that has latitude > 50 or < -50
# Filter out the alerts that are before the year of 2000
CCHF_df_test = pd.read_excel('/Users/xiongh/Documents/Codes/Data_Analytics/Final_Project/CCHF - Total ProMED Archive.xlsx')
CCHF_df_test['lat']  = CCHF_df_test['lat'].apply(lambda x: np.floor(x).astype(int))
CCHF_df_test['lon']  = CCHF_df_test['lon'].apply(lambda x: np.floor(x).astype(int))
#CCHF_df_test['issue_date'] = CCHF_df_test['issue_date'].apply(lambda x: datetime.datetime(x.year,x.month,x.day))
CCHF_df_test['time'] = CCHF_df_test['issue_date'].apply(lambda x: x.strftime("%m/%d/%Y"))
CCHF_df_test = CCHF_df_test.loc[CCHF_df_test['time'].apply(lambda x: int(x[-4:]) >= 2000)]
CCHF_df_test.reset_index(inplace=True,drop=True)
CCHF_df_test['coor'] = CCHF_df_test.apply(lambda x: (x['lon'],x['lat']),axis = 1)
location_set = set(CCHF_df_test['coor'])
unique_location = set()
for i in location_set:
    if i[1] < 50 and i[1] > -50:
        unique_location.add(i)
CCHF_df_test = CCHF_df_test.loc[CCHF_df_test['coor'].isin(unique_location)]

In [ ]:
len(CCHF_df_test),len(unique_location)

(899, 134)

In [ ]:
#Select the subset of df based on location
#df_test = nc4_to_df('/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20020915.7.nc4')
df_test = pd.read_csv('/Users/xiongh/Documents/Codes/Data_Analytics/ori_a.csv')
df_test['lat']  = df_test['lat'].apply(lambda x: np.floor(x).astype(int))
df_test['lon']  = df_test['lon'].apply(lambda x: np.floor(x).astype(int))
# df_test.fillna(0,inplace=True)
#df_test['precip'] = df_test['precip'].ffill().bfill()
#df_test['time'] = pd.to_datetime(df_test['time'])
df_test['coor'] = df_test.apply(lambda x: (x['lon'],x['lat']),axis = 1)
loc_index = df_test.loc[df_test['coor'].isin(unique_location)].index
df_test = df_test.iloc[loc_index]
# this_df.reset_index(inplace=True,drop=True)


,Unnamed: 0,lat,lon,precip,time,coor
150552,150552,-34,18,0.0,02/12/2013,"(18, -34)"
150553,150553,-34,18,0.0,02/12/2013,"(18, -34)"
150554,150554,-34,18,0.0,02/12/2013,"(18, -34)"
150555,150555,-34,18,0.0,02/12/2013,"(18, -34)"
151992,151992,-34,18,0.0,02/12/2013,"(18, -34)"
...,...,...,...,...,...,...
631615,631615,49,43,0.0,02/12/2013,"(43, 49)"
633052,633052,49,43,0.0,02/12/2013,"(43, 49)"
633053,633053,49,43,0.0,02/12/2013,"(43, 49)"
633054,633054,49,43,0.0,02/12/2013,"(43, 49)"


In [ ]:
na_index = list(df_blocks.loc[(df_blocks.precip.isna() == True)].index)

In [ ]:
def fill_na(index_0, df_ori,fillna_dict):
    rad = 0
    na_ratio = 0
    radius_dict = dict()
    cor_dict = dict()
    start = time.time()
    not_na_count = 0

    #Find the radius needed to achieve 1/3 of non values
    #Store the coordinates used and their precipitations
    while na_ratio < 1/3:
        rad += 1
        itr_list = [0]
        itr_list.extend([i for i in range(1,rad+1)])
        itr_list.extend([-i for i in range(1,rad+1)])
        x_itr = [i for i in range(-rad,rad+1)]
        y_itr = [i for i in range(-rad+1,rad)]
        cor_itr = list()
        for x in x_itr: #Fix y at y+rad and y-rad, then itr through all x from x-rad to x+rad
            cor_itr.append((index_0[0]+x,index_0[1]+rad))
            cor_itr.append((index_0[0]+x,index_0[1]-rad))
        for y in y_itr: #Fix x at x+rad and x-rad, then itr through all y from y-rad to y+rad besides corners
            cor_itr.append((index_0[0]+rad,index_0[1]+y))
            cor_itr.append((index_0[0]-rad,index_0[1]+y))

        precip_list = list()
        cor_list = list()
        for i in cor_itr:
            _precip = df_ori.loc[(i[0],i[1])]
            if not (_precip.isna()[0]):
                precip_list.append(_precip[('precip', 'mean')])
                not_na_count += 1
                cor_list.append((i[0],i[1]))


        cor_dict[rad] = cor_list
        radius_dict[rad] = precip_list  
        na_ratio = not_na_count/((2*rad+1)**2)

    #Find non empty radius    
    not_empty_rad_list = list()
    for i,j in radius_dict.items():
        if len(j):
            not_empty_rad_list.append(i)

    #Find the weight for each radius
    k = 0
    n = len(not_empty_rad_list)
    not_empty_rad_list.sort()
    fill_value = 0
    weight_dict = dict()
    for i in not_empty_rad_list:
        k += 1
        weight = ((2**(n-k))/(2**(n)-1))
        fill_value = np.mean(radius_dict[i])*weight
        weight_dict[i] = weight
    # Put the needed coordinate and their weights in a single dictionary
    fillna_dict[index_0] = dict()
    for i,j in weight_dict.items():
        fillna_dict[index_0][i] = (cor_dict[i],j)

    end= time.time()
#     print(end - start,'s')
    return fillna_dict

In [ ]:
# fillna_dict - key: na coordinate, value: another dictionary
# another dictionary - key: radius value: tuple (all coordinates needed, weight)
for i in na_index: 
    fillna_dict = fill_na(i, df_ori,fillna_dict)

In [ ]:
fillna_dict

{(20,
  56): {7: ([(13, 49),
    (14, 49),
    (15, 49),
    (16, 49),
    (17, 49),
    (18, 49),
    (19, 49),
    (20, 49),
    (21, 49),
    (22, 49),
    (23, 49),
    (24, 49),
    (25, 49),
    (26, 49),
    (27, 49)],
   0.5009784735812133), 8: ([(12, 48),
    (13, 48),
    (14, 48),
    (15, 48),
    (16, 48),
    (17, 48),
    (18, 48),
    (19, 48),
    (20, 48),
    (21, 48),
    (22, 48),
    (23, 48),
    (24, 48),
    (25, 48),
    (26, 48),
    (27, 48),
    (28, 48),
    (28, 49),
    (12, 49)],
   0.25048923679060664), 9: ([(11, 47),
    (12, 47),
    (13, 47),
    (14, 47),
    (15, 47),
    (16, 47),
    (17, 47),
    (18, 47),
    (19, 47),
    (20, 47),
    (21, 47),
    (22, 47),
    (23, 47),
    (24, 47),
    (25, 47),
    (26, 47),
    (27, 47),
    (28, 47),
    (29, 47),
    (29, 48),
    (11, 48),
    (29, 49),
    (11, 49)],
   0.12524461839530332), 10: ([(10, 46),
    (11, 46),
    (12, 46),
    (13, 46),
    (14, 46),
    (15, 46),
    (16, 46),
    (17,

In [ ]:
# Fill value
df_blocks.loc[index_0,'precip'] = fill_value

In [ ]:
df_blocks

precip
lat lon          
-34 18   0.000000
-33 26   0.001875
-31 21   0.000000
-30 22   0.000000
-29 25   0.000000
...           ...
 47 42   0.000000
 48 41   0.063750
    44   0.352500
    66        NaN
 49 43   0.000000

[134 rows x 1 columns]

In [ ]:
directory = r'/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution'
final_df = pd.DataFrame()
very_start = time.time()
for filename in os.listdir(directory):
    if filename.endswith(".nc4"):
        this_df = nc4_to_df(os.path.join(directory, filename))
        print(os.path.join(directory, filename))
        this_df['lat']  = this_df['lat'].apply(lambda x: np.floor(x).astype(int))
        this_df['lon']  = this_df['lon'].apply(lambda x: np.floor(x).astype(int))
        df_ori = this_df.groupby(['lat','lon']).agg({'precip': ['mean']})
        this_time = this_df['time']
        
        this_df = this_df.iloc[loc_index]
        this_blocks = this_df.groupby(['lat','lon']).agg({'precip': ['mean']})
        this_blocks = this_blocks.droplevel(axis=1,level=1)
        for i in fillna_dict.keys():
            _value = 0
            for j,k in fillna_dict[i].items():
                mean_precip = np.mean(df_ori.loc[k[0]].precip)
                mean_precip = float(mean_precip)
                weight = k[1]
                _value += mean_precip * weight
            this_blocks.loc[i] = _value
        this_blocks = this_blocks.reset_index()
        this_blocks['time'] = pd.Series([this_time[0]]*len(df_blocks))
        sb_df = this_blocks
        
        
        final_df = final_df.append(this_blocks)
    else:
        continue
very_end = time.time()
final_df.to_csv('final_csv.csv')
print("Concatanation Complete!")
print('Total time ellapsed:',round(very_end - very_start,2),'s')

# fillna_dict - key: na coordinate, value: another dictionary
# another dictionary - key: radius value: tuple (all coordinates needed, weight)

/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000301.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000302.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000303.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000304.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000305.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000306.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000307.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000308.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000309.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000310.7.nc4


/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000522.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000523.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000524.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000525.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000526.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000527.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000528.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000529.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000530.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000531.7.nc4


/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000812.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000813.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000814.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000815.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000816.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000817.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000818.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000819.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000820.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20000821.7.nc4


/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20001102.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20001103.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20001104.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20001105.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20001106.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20001107.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20001108.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20001109.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20001110.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20001111.7.nc4


/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20010123.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20010124.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20010125.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20010126.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20010127.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20010128.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20010129.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20010130.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20010131.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20010201.7.nc4


...

/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190109.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190110.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190111.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190112.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190113.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190114.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190115.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190116.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190117.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190118.7.nc4


/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190401.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190402.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190403.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190404.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190405.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190406.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190407.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190408.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190409.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190410.7.nc4


/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190622.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190623.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190624.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190625.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190626.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190627.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190628.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190629.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190630.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190701.7.nc4


/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190912.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190913.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190914.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190915.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190916.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190917.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190918.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190919.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190920.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20190921.7.nc4


/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20191203.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20191204.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20191205.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20191206.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20191207.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20191208.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20191209.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20191210.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20191211.7.nc4
/Volumes/TOSHIBA EXT/Data_Analytics_Data/Daily_GPM_Data_0.25_Resolution/3B42RT_Daily.20191212.7.nc4


In [ ]:
%ls

Applications/  Downloads/     Movies/        Public/
Desktop/       Google Drive@  Music/         final_csv.csv
Documents/     Library/       Pictures/


In [ ]:
na_index

[(20, 56),
 (23, 54),
 (24, 55),
 (24, 56),
 (24, 66),
 (24, 67),
 (25, 55),
 (25, 69),
 (26, 67),
 (26, 68),
 (28, 66),
 (28, 69),
 (29, 60),
 (29, 68),
 (29, 69),
 (30, 67),
 (30, 68),
 (31, 68),
 (32, 54),
 (33, 66),
 (34, 62),
 (36, 59),
 (41, 63),
 (43, 68),
 (44, 65),
 (44, 66),
 (48, 66)]

In [ ]:
final_df.loc[(final_df.precip.isna())]

,lat,lon,precip,time


In [ ]:
final_df = final_df.ffill()

In [ ]:
final_df['multiIndex'] = pd.MultiIndex.from_arrays([final_df['time'],final_df['lat'],final_df['lon']])

In [ ]:
final_df

,lat,lon,precip,time,multiIndex
0,-34,18,0.000000,03/01/2000,"(03/01/2000, -34, 18)"
1,-33,26,0.065625,03/01/2000,"(03/01/2000, -33, 26)"
2,-31,21,0.000000,03/01/2000,"(03/01/2000, -31, 21)"
3,-30,22,0.026250,03/01/2000,"(03/01/2000, -30, 22)"
4,-29,25,0.033750,03/01/2000,"(03/01/2000, -29, 25)"
...,...,...,...,...,...
129,47,42,0.000000,12/31/2019,"(12/31/2019, 47, 42)"
130,48,41,0.000000,12/31/2019,"(12/31/2019, 48, 41)"
131,48,44,0.112500,12/31/2019,"(12/31/2019, 48, 44)"
132,48,66,2.415755,12/31/2019,"(12/31/2019, 48, 66)"


In [ ]:
CCHF_df_test = CCHF_df_test.iloc[7:]

In [ ]:
CCHF_df_test = CCHF_df_test.iloc[:884]

In [ ]:
CCHF_df_test.reset_index(inplace=True,drop=True)

In [ ]:
len(CCHF_df_test)

884

In [ ]:
# Fill target_label based on CCHF excel file
CCHF_df_test['multiIndex'] = pd.MultiIndex.from_arrays([CCHF_df_test['time'],CCHF_df_test['lat'],CCHF_df_test['lon']])
final_df_with_label = final_df.copy()
final_df_with_label['target_label'] = pd.Series([0]*len(final_df_with_label))
final_df_with_label = final_df_with_label.set_index(['time','lat','lon']) 
for i in range(len(CCHF_df_test)):
    final_df_with_label.loc[CCHF_df_test.loc[i,'multiIndex'], ['target_label']] += 1
# final_df['target_label'] = final_df['target_label'].astype(int)

<ipython-input-505-6bc3260ae192>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CCHF_df_test['multiIndex'] = pd.MultiIndex.from_arrays([CCHF_df_test['time'],CCHF_df_test['lat'],CCHF_df_test['lon']])


In [ ]:
final_df_with_label.loc[final_df_with_label.target_label == 2]

precip             multiIndex  target_label
time       lat lon                                                
09/25/2000  29 69    0.000000   (09/25/2000, 29, 69)             2
11/08/2000  29 69    0.051875   (11/08/2000, 29, 69)             2
02/14/2001 -29 25    0.000000  (02/14/2001, -29, 25)             2
06/16/2001  43 20    9.315001   (06/16/2001, 43, 20)             2
06/15/2008  39 32    0.000000   (06/15/2008, 39, 32)             2
06/20/2008  46 48    0.000000   (06/20/2008, 46, 48)             2
            48 44    2.820000   (06/20/2008, 48, 44)             2
07/23/2008  47 42    0.421875   (07/23/2008, 47, 42)             2
09/14/2013  34 73    0.000000   (09/14/2013, 34, 73)             2
10/11/2014  24 67    1.685015   (10/11/2014, 24, 67)             2
05/15/2015  29 69    0.000000   (05/15/2015, 29, 69)             2
09/30/2015  24 67    0.128952   (09/30/2015, 24, 67)             2
10/07/2015  29 69    0.000000   (10/07/2015, 29, 69)             2
02/19/2016  23 72    0.000000   (02/19/2016, 23, 72)             2
07/31/2016  31 72    3.399375   (07/31/2016, 31, 72)             2
08/26/2016  29 69    0.417500   (08/26/2016, 29, 69)             2
09/03/2016  29 69    0.000000   (09/03/2016, 29, 69)             2
09/13/2016  33 66    0.095276   (09/13/2016, 33, 66)             2
09/17/2016  28 66    0.338255   (09/17/2016, 28, 66)             2
09/30/2016  21 72    2.520000   (09/30/2016, 21, 72)             2
11/01/2016  26 68    1.539320   (11/01/2016, 26, 68)             2
03/17/2017  30 71    4.430625   (03/17/2017, 30, 71)             2
05/08/2017  30 68    0.932095   (05/08/2017, 30, 68)             2
06/08/2017  32 54    3.330827   (06/08/2017, 32, 54)             2
07/06/2017  28 66    0.313937   (07/06/2017, 28, 66)             2
08/06/2017  31 72    0.000000   (08/06/2017, 31, 72)             2
08/30/2017  28 66    0.029965   (08/30/2017, 28, 66)             2
09/04/2017  34 62    0.344102   (09/04/2017, 34, 62)             2
09/16/2017  29 69    0.000000   (09/16/2017, 29, 69)             2
09/23/2017  26 68    0.033565   (09/23/2017, 26, 68)             2
09/24/2017  26 68    1.178003   (09/24/2017, 26, 68)             2
10/08/2017  30 67    0.329773   (10/08/2017, 30, 67)             2
10/22/2017  29 69    1.648750   (10/22/2017, 29, 69)             2
11/02/2017  33 71    6.840000   (11/02/2017, 33, 71)             2
11/12/2017  22 72    0.000000   (11/12/2017, 22, 72)             2
12/23/2017  33 66    0.489647   (12/23/2017, 33, 66)             2
04/24/2018  30 71   11.578125   (04/24/2018, 30, 71)             2
04/27/2018  31 72   10.357500   (04/27/2018, 31, 72)             2
05/06/2018  24 67    3.857874   (05/06/2018, 24, 67)             2
            32 54    0.953066   (05/06/2018, 32, 54)             2
05/10/2018  26 75    3.470625   (05/10/2018, 26, 75)             2
05/26/2018  28 66    0.011192   (05/26/2018, 28, 66)             2
06/03/2018  32 54    2.249960   (06/03/2018, 32, 54)             2
06/28/2018  33 66    0.154178   (06/28/2018, 33, 66)             2
07/31/2018  32 54    7.397390   (07/31/2018, 32, 54)             2
08/09/2018  33 73    1.923750   (08/09/2018, 33, 73)             2
08/21/2018  31 72    0.600000   (08/21/2018, 31, 72)             2
09/04/2018  34 72    0.000000   (09/04/2018, 34, 72)             2
09/06/2018  34 72    2.163750   (09/06/2018, 34, 72)             2
09/12/2018  26 68    2.842397   (09/12/2018, 26, 68)             2
09/22/2018  26 68    1.735587   (09/22/2018, 26, 68)             2
11/10/2018  29 69    0.651250   (11/10/2018, 29, 69)             2
12/04/2018  11 76    3.519375   (12/04/2018, 11, 76)             2
            23 54    3.182658   (12/04/2018, 23, 54)             2
12/10/2018  34 72    0.000000   (12/10/2018, 34, 72)             2
07/04/2019  24 67    0.979429   (07/04/2019, 24, 67)             2
07/09/2019  34 72    0.862500   (07/09/2019, 34, 72)             2
11/13/2019  24 67    0.008748   (11/13/2019, 24, 67)             2

In [ ]:
a = final_df_with_label.copy()

In [ ]:
final_df_with_label = final_df_with_label.set_index(['time','lat','lon']) 

In [ ]:
final_df_with_label['target_label'] = pd.Series([0]*len(final_df_with_label))

In [ ]:
final_df_with_label

precip             multiIndex  target_label
time       lat lon                                               
03/01/2000 -34 18   0.000000  (03/01/2000, -34, 18)             0
           -33 26   0.065625  (03/01/2000, -33, 26)             0
           -31 21   0.000000  (03/01/2000, -31, 21)             0
           -30 22   0.026250  (03/01/2000, -30, 22)             0
           -29 25   0.033750  (03/01/2000, -29, 25)             0
...                      ...                    ...           ...
12/31/2019  47 42   0.000000   (12/31/2019, 47, 42)             0
            48 41   0.000000   (12/31/2019, 48, 41)             0
               44   0.112500   (12/31/2019, 48, 44)             0
               66   2.415755   (12/31/2019, 48, 66)             0
            49 43   1.428750   (12/31/2019, 49, 43)             0

[970830 rows x 3 columns]

In [ ]:
final_df_with_label = final_df_with_label.drop('multiIndex',axis = 1)

In [ ]:
final_df_with_label.to_csv('final_csv.csv')

In [ ]:
final_df_with_label.target_label.value_counts()

0    970008
1       762
2        58
3         2
Name: target_label, dtype: int64

In [ ]:
sum(final_df_with_label.target_label == 2)

58

In [ ]:
sum(final_df_with_label.target_label == 3)

2

In [ ]:
sum(final_df_with_label.target_label == 4)

0

In [ ]:
len(CCHF_df_test)

899